# Build
---
This notebook builds Cirrus. It is mainly intended for use by Cirrus developers.

## Setup
---

In [ ]:
# To ease development, each time a cell is run, all modules will be reloaded.
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import sys
import atexit
import subprocess

from cirrus import automate
from cirrus import setup

In [ ]:
# Cirrus produces logs, but they will not show unless we add a handler that prints.
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("[%(funcName)16s | %(threadName)15s] %(message)s")
handler.setFormatter(formatter)
logger = logging.getLogger("cirrus")
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

## Ubuntu
---
Cirrus needs to be built under Ubuntu in order to create an executable for the parameter server.

### Build image
First, we create a "build image", which is an AMI on which we've set up the proper environment for building Cirrus. Making the build image is separated from making the executables because the build image will rarely need to be remade, whereas the executables will need to be remade for each new version of the system.

In [ ]:
automate.make_ubuntu_build_image("cirrus_ubuntu_build_image")

### Executables
Second, we compile Cirrus (using the build image) and publish its executables.

The executables are published to the `cirrus-public` S3 bucket. Only Joao's AWS account can write to it.

In [ ]:
automate.make_executables(
    setup.PUBLISHED_BUILD + "/executables/ubuntu",
    ("self", "cirrus_ubuntu_build_image"),
    "ubuntu"
)

## Amazon Linux
---
Cirrus needs to be built under Amazon Linux in order to create an executable for the workers.

### Build image
See Ubuntu#Build image.

In [ ]:
automate.make_amazon_build_image("cirrus_amazon_build_image")

### Executables
See Ubuntu#Executables.

In [ ]:
automate.make_executables(
    setup.PUBLISHED_BUILD + "/executables/amazon",
    ("self", "cirrus_amazon_build_image"),
    "ec2-user"
)

The Amazon Linux compilation process currently does not work properly, so replace the `parameter_server` executable with a correctly-compiled version.

In [ ]:
subprocess.check_call(
    "; ".join((
        "wget https://www.dropbox.com/s/dr6g1vssjgc02d4/parameter_server_18-11-28?dl=0 -O parameter_server_18-11-28",
        "aws s3 cp parameter_server_18-11-28 %s/executables/amazon/parameter_server" % setup.PUBLISHED_BUILD,
        "rm parameter_server_18-11-28"
    )),
    shell=True
)

# Cleanup
---
If a cell errors, running this should clean up any resources that were created. After running this cell, the kernel will become unusable and need to be restarted.

In [ ]:
atexit._run_exitfuncs()